In [1]:
import numpy as np
import scipy as scp
import pylab as pyl
import pandas as pd
import bokeh as bk
import bokeh.plotting as bkp

bkp.output_notebook()
#import pandas_bokeh
#pandas_bokeh.output_notebook()

Loading BokehJS ...

# ADEME_S1 à ADEME_S4
Interpolation linéaire des scénaries de 2025 à 2050 par pas de 5 ans

## ADEME 2050 S1 à S4

In [2]:
def cumul_plot(T, cols, palette=bk.palettes.Category10_10):
    couleurs = iter(palette)
    source = bk.models.ColumnDataSource(T)
    fig = bkp.figure()
    fig.varea_stack(stackers=cols, x="Horodatage", color=[next(couleurs) for col in cols], legend_label=cols, source=source )
    fig.legend.click_policy = "mute"    
    bkp.show(fig)


In [3]:
ADEME2050 = pd.read_csv("chroniques-horaires-de-la-consommation-d'electricite.csv")

ADEME2050.rename({ADEME2050.columns[4]: "e2050"}, axis=1, inplace=True)
ADEME2050["e2050"] = pd.to_numeric(ADEME2050["e2050"].str.replace(',', '.'))
print(ADEME2050.columns)
ADEME2050.insert(loc=4, column='d2050', value=ADEME2050.iloc[:,[2, 3]].sum(axis=1))
print(ADEME2050.columns)

ADEME2050.drop(columns=ADEME2050.columns[[2,3]], inplace=True)
ADEME2050.set_index(['Horodatage'], inplace=True)
#pd.to_datetime(ADEME2050[["Horodatage"]])
#ADEME2050.set_index(["Scénario","Horodatage"], inplace=True)
#print(ADEME2050.head())
#pd.to_datetime(ADEME2050[["Horodatage"]]).day*24 + pd.to_datetime(ADEME2050[["Horodatage"]]).hour 
#ADEME2050.drop(columns=["Horodatage"], inplace=True)
#ADEME2050.insert(loc=0, column="heures", 
 #                value=T1["heures"].tolist() * len(ADEME2050["Scénario"].unique()))
#ADEME2050.groupby("Scénario").describe()

Index(['Scénario', 'Horodatage', 'Demande directe non pilotable',
       'Demande directe pilotable', 'e2050'],
      dtype='object')
Index(['Scénario', 'Horodatage', 'Demande directe non pilotable',
       'Demande directe pilotable', 'd2050', 'e2050'],
      dtype='object')


In [4]:
for nom, group in ADEME2050.groupby("Scénario"):
    print(nom)
    print(group.describe())
    print(group.index.is_monotonic_increasing)
    #cumul_plot(group, cols=group.columns.tolist()[1:])

S1
             d2050         e2050
count  8760.000000  8.760000e+03
mean     38.927280  7.909959e+00
std      11.199089  1.069097e+01
min      17.852749  1.023440e-10
25%      30.655466  4.088539e-01
50%      37.851607  4.090182e-01
75%      45.292254  1.765474e+01
max      91.609838  2.720755e+01
True
S2
             d2050         e2050
count  8760.000000  8.760000e+03
mean     44.057864  1.600812e+01
std      11.400370  1.579146e+01
min      20.998007  2.769410e-10
25%      35.293158  5.757742e+00
50%      43.459005  8.779163e+00
75%      51.174528  2.801626e+01
max      94.342464  4.582176e+01
True
S3Enr
             d2050         e2050
count  8760.000000  8.760000e+03
mean     66.277096  8.789603e+00
std      17.171094  6.724212e+00
min      30.544315  1.824650e-10
25%      53.442241  5.536506e+00
50%      64.938216  6.561460e+00
75%      77.492507  1.769360e+01
max     140.554904  1.818649e+01
True
S3Nuke
             d2050         e2050
count  8760.000000  8.760000e+03
mean     

## ADEME 2025 (fichier marion)

In [13]:
fich = "ADEME_25-50_marion.csv"
df2025 = pd.read_csv(fich, header=None)
df2025.columns = ["heures", "d2025", "d2030", "d2035", "d2040", "d2045", "d2050" ]
df2025.set_index(["heures"], inplace=True)
print(df2025.index.is_monotonic_increasing)
print(df2025.describe())
#scenar2025 = df2025[["heures", "d2025"]]
df2025.insert(loc= 2, column="e2025", value=df2025["d2025"] * 0.)

print("Demande en 2025")
print(df2025.describe())
print(df2025["d2025"].sum())




True
             d2025        d2030        d2035        d2040        d2045  \
count  8760.000000  8760.000000  8760.000000  8760.000000  8760.000000   
mean     48.216612    49.180944    50.145277    51.109609    52.073941   
std      13.492965    13.762824    14.032684    14.302543    14.572402   
min      16.833074    17.169735    17.506396    17.843058    18.179719   
25%      37.890636    38.648448    39.406261    40.164074    40.921886   
50%      48.344286    49.311171    50.278057    51.244943    52.211829   
75%      57.788214    58.943978    60.099742    61.255507    62.411271   
max      96.041075    97.961896    99.882718   101.803539   103.724361   

             d2050  
count  8760.000000  
mean     53.038273  
std      14.842262  
min      18.516381  
25%      41.679699  
50%      53.178714  
75%      63.567035  
max     105.645183  
Demande en 2025
             d2025        d2030   e2025        d2035        d2040  \
count  8760.000000  8760.000000  8760.0  8760.000000  

## Interpolation de 2025 à 2050

In [6]:

scenars={}
for nom, group in ADEME2050.groupby("Scénario"):
    print(nom)
    group.insert(loc=1, column="heures", value=df2025.index.tolist())
    for annee in range(2045,2020,-5):
        alpha = float(annee-2025.)/float(2050-2025)
        for dem_elec in ["e","d"]:
            liste50 = group[dem_elec+"2050"].values
            liste25 = df2025[dem_elec+"2025"].values
            group.insert(loc=2, 
                         column = dem_elec+annee.__str__(), 
                         value = [a*(1.-alpha) + b*alpha for a, b in zip(liste25, liste50)]
                        )
    scenars[nom]=group.drop(columns="Scénario")
    print(group.describe()) 
    print("======================================================")

    scenars[nom].to_csv(nom+"_25-50.csv", index=False)

S1
            heures        d2025   e2025        d2030         e2030  \
count  8760.000000  8760.000000  8760.0  8760.000000  8.760000e+03   
mean   4379.500000    48.216612     0.0    46.358746  1.581992e+00   
std    2528.938512    13.492965     0.0    12.439670  2.138194e+00   
min       0.000000    16.833074     0.0    18.580090  2.046880e-11   
25%    2189.750000    37.890636     0.0    36.891770  8.177078e-02   
50%    4379.500000    48.344286     0.0    46.314959  8.180365e-02   
75%    6569.250000    57.788214     0.0    54.754274  3.530948e+00   
max    8759.000000    96.041075     0.0    94.209661  5.441511e+00   

             d2035         e2035        d2040         e2040        d2045  \
count  8760.000000  8.760000e+03  8760.000000  8.760000e+03  8760.000000   
mean     44.500879  3.163984e+00    42.643013  4.745975e+00    40.785146   
std      11.637180  4.276388e+00    11.139832  6.414583e+00    10.989134   
min      20.323342  4.093760e-11    20.406492  6.140640e-11   

In [7]:
def une_annee_df(df, annee):
    longdf = df[["heures"]]
    longdf.insert(loc=1, column='date', value= pd.to_datetime(annee+"-01-01") + pd.to_timedelta(df['heures'], unit='h'))
    longdf.insert(loc=2, column='demande', value= df["d"+annee].values)
    longdf.insert(loc=3, column='electrolyse', value= df["e"+annee].values)
    return longdf

In [8]:
import os
mix_data="./"
filenames = os.listdir(mix_data)
fichiers = [ filename for filename in filenames if filename.endswith("_25-50.csv") ]

for fichier in fichiers:
    print(fichier)
    dfinit = pd.read_csv(mix_data+fichier)
    if 'e2025' not in dfinit.columns:
        dfinit.insert(1,'e2025',dfinit["heures"]*0.)
           
    df = une_annee_df(dfinit,'2025')
    zeros = dfinit["heures"]*0.
    for column in dfinit.columns:
        annee = column[1:]
        if 'e'+annee not in dfinit.columns:
            dfinit.insert(1,'e'+annee,zeros)
        if column[0]=='d' and annee != "2025":
            df = pd.concat([df, une_annee_df(dfinit,annee)], ignore_index= True)     
    df.drop(columns=["heures"], inplace=True)
    df.set_index(["date"], inplace=True)
    print(df.describe())         
    df.to_hdf(mix_data+fichier[:-4]+'.h5', key='df', mode='w')  
    #os.remove(mix_data+fichier)
    

S1_25-50.csv
            demande   electrolyse
count  52560.000000  5.256000e+04
mean      43.571946  3.954979e+00
std       12.266697  7.014595e+00
min       16.833074  0.000000e+00
25%       34.407489  1.661394e-08
50%       42.697962  2.453123e-01
75%       51.293406  5.061730e+00
max       96.041075  2.720755e+01
S3Enr_25-50.csv
            demande   electrolyse
count  52560.000000  5.256000e+04
mean      57.246854  4.394801e+00
std       15.885599  5.058732e+00
min       16.833074  0.000000e+00
25%       45.857394  2.395662e-08
50%       56.353004  2.657668e+00
75%       67.001892  6.561460e+00
max      140.554904  1.818649e+01
S4_25-50.csv
            demande   electrolyse
count  52560.000000  5.256000e+04
mean      69.261217  2.144641e+00
std       22.472518  2.311619e+00
min       16.833074  0.000000e+00
25%       53.213445  1.778220e-08
50%       66.883838  1.602644e+00
75%       83.022974  3.280897e+00
max      178.214636  8.013221e+00
S2_25-50.csv
            demande   elect

In [9]:
 df.to_hdf(mix_data+fichier[:-4]+'.h5', key='df', mode='w')  

## Déploiement

Copier tout les _25-50.h5 dans ../  soit /climix/mix_data